In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
train=pd.read_csv('../dataset/train.csv')
test=pd.read_csv('../dataset/test.csv')
train.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [3]:
#Enumerating classes

Class_map={'Food':0,'Attire':1,'Decorationandsignage':2,'misc':3}
inverse_map={0:'Food',1:'Attire',2:'Decorationandsignage',3:'misc'}
train['Class']=train['Class'].map(Class_map)

In [4]:
train_img=[]
train_label=[]
j=0
path='../dataset/TrainImages'
for i in tqdm(train['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(150,150))
    img=img.astype('float32')
    train_img.append(img)
    train_label.append(train['Class'][j])
    j=j+1

100%|██████████| 5983/5983 [00:15<00:00, 397.53it/s]


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(train_img)

In [7]:
test_img=[]
path='../dataset/TestImages'
for i in tqdm(test['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(150,150))
    img=img.astype('float32')
    test_img.append(img)

100%|██████████| 3219/3219 [00:07<00:00, 426.35it/s]


In [ ]:
datagen.fit(test_img)

In [8]:
train_img=np.array(train_img)
test_img=np.array(test_img)
train_label=np.array(train_label)

In [15]:
train_img.shape

(5983, 150, 150, 3)

In [9]:
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from tensorflow.keras.callbacks import ReduceLROnPlateau

In [21]:
base_model=VGG16(include_top=False, weights='imagenet',input_shape=(150,150,3), pooling='avg')

model=Sequential()
model.add(base_model)

model.add(Dense(256,activation='relu'))
model.add(Dense(4,activation='softmax'))


from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

base_model.trainable=False

reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)

callbacks = [reduce_learning_rate]
    


model.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(datagen.flow(train_img, to_categorical(train_label,4), batch_size=64),
                    epochs=10,callbacks=callbacks)

  ...
    to  
  ['...']
Train for 94 steps
Epoch 1/10
94/94 [==============================] - 792s 8s/step - loss: 1.8787 - accuracy: 0.6209
Epoch 2/10
94/94 [==============================] - 688s 7s/step - loss: 0.8841 - accuracy: 0.7251
Epoch 3/10
94/94 [==============================] - 680s 7s/step - loss: 0.6797 - accuracy: 0.7491
Epoch 4/10
94/94 [==============================] - 683s 7s/step - loss: 0.5782 - accuracy: 0.7901
Epoch 5/10
94/94 [==============================] - 742s 8s/step - loss: 0.5176 - accuracy: 0.8011
Epoch 6/10
94/94 [==============================] - 772s 8s/step - loss: 0.4634 - accuracy: 0.8230
Epoch 7/10
94/94 [==============================] - 1210s 13s/step - loss: 0.4318 - accuracy: 0.8345
Epoch 8/10
94/94 [==============================] - 1079s 11s/step - loss: 0.3741 - accuracy: 0.8504
Epoch 9/10
94/94 [==============================] - 694s 7s/step - loss: 0.3644 - accuracy: 0.8603
Epoch 10/10
94/94 [==============================] - 729s 8s/

In [22]:
labels = model.predict(test_img)
print(labels[:4])
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
print(class_label[:3])
submission = pd.DataFrame({ 'Image': test.Image, 'Class': class_label })
submission.head(10)
# submission.to_csv('Users/megharana/Downloads/dataset/testHighAcc.csv', index=False)


[[9.6416938e-01 7.1623549e-03 1.6608264e-03 2.7007455e-02]
 [7.2797936e-01 6.7317002e-03 1.8740208e-04 2.6510161e-01]
 [9.2613506e-01 1.0038233e-11 1.0566367e-12 7.3864929e-02]
 [9.9990976e-01 3.8648414e-11 1.0973141e-09 9.0257323e-05]]
['Food', 'Food', 'Food']


,Image,Class
0,image6245.jpg,Food
1,image10409.jpg,Food
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Attire
5,image848.jpg,Attire
6,image1232.jpg,Attire
7,image8386.jpg,misc
8,image9828.jpg,misc
9,image9033.jpg,Attire


In [23]:
submission

,Image,Class
0,image6245.jpg,Food
1,image10409.jpg,Food
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Attire
...,...,...
3214,image4968.jpg,Attire
3215,image6673.jpg,Food
3216,image3442.jpg,misc
3217,image8178.jpg,misc


In [25]:
submission.to_csv('/Users/megharana/Downloads/dataset/test5.csv', index=False)